<h1>
<center>
Assignment 1: Wrangle that data
</center>
</h1>

I'll ask you to use the skills you learned working with the Titanic table on a new table. The new table is data collected by a bank on the loans the bank gave out over a short period. There are 614 seperate loan applications. Some were accepted ("survived" in Titanic terms) and some were rejected ("died"). Eventually we will explore several machine learning models that try to learn how to recognize winning applications from losing applications. For now, we are just trying to wrangle the data we recieve.
<p>
You will see the general style I use for homeworks (and exams) is to solve the problem first myself, then remove the code and leave you with the output. So you can see what you should produce. Your job is to fill the code back in that produces it.

# Read in spreadsheet

For the loan table, I stored the csv file in google sheets. I got its url and pasted it below. That's all I had to do. In particular, no need to copy/upload the csv file to colab. Just read it directly off the cloud.

In [32]:
import pandas as pd
url = 'https://docs.google.com/spreadsheets/d/1_artlzgoj6pDBCBfdt9-Jmc9RT9yLsZ0vTnk3zJmt_E/pub?gid=1291197392&single=true&output=csv'

loan_table = pd.read_csv(url)
len(loan_table)  #614

614

In [0]:
#I am setting the option to see all the columns of our table as we build it.
pd.set_option('display.max_columns', None)

# 1. Explore

* Use head to get general layout.

* Find which columns have NaNs (empties) and how many

* Use describe method to see if any odd looking columns, e.g., more than 2 unqiue values for a binary column

In [34]:
#show first 5 rows
loan_table.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [35]:
#get a count of empties in each column
print(loan_table.isnull().sum())

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64


In [36]:
#show statistics on each column
loan_table.describe(include='all')

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
count,614,601,611,599,614,582,614.000000,614.000000,592.000000,600.00000,564.000000,614,614
unique,614,2,2,4,2,2,NaN,NaN,NaN,NaN,NaN,3,2
top,LP001964,Male,Yes,0,Graduate,No,NaN,NaN,NaN,NaN,NaN,Semiurban,Y
freq,1,489,398,345,480,500,NaN,NaN,NaN,NaN,NaN,233,422
mean,NaN,NaN,NaN,NaN,NaN,NaN,5403.459283,1621.245798,146.412162,342.00000,0.842199,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,6109.041673,2926.248369,85.587325,65.12041,0.364878,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,150.000000,0.000000,9.000000,12.00000,0.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,2877.500000,0.000000,100.000000,360.00000,1.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,3812.500000,1188.500000,128.000000,360.00000,1.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,5795.000000,2297.250000,168.000000,360.00000,1.000000,NaN,NaN


# 2. Drop the Loan_ID column

It does not carry useful information.

In [0]:
#drop the column
loan_table = loan_table.drop(['Loan_ID'], axis=1)

In [38]:
loan_table.head(1)  # Should see drop of Loan_ID column

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y


# 3. Outliers

Compute 3-sigma for the `LoanAmount` column. You need low and high values.

Then show all rows lower than low value and higher than high value.

In [39]:
sigma3_lam =  loan_table['LoanAmount'].std() * 3 # calculate std * 3
mean_lam =   loan_table['LoanAmount'].mean()# calculate mean
low_lam = mean_lam - sigma3_lam
high_lam = mean_lam + sigma3_lam

print((low_lam,high_lam))   # (-110.34981354495417, 403.1741378692785)

(-110.34981354495417, 403.1741378692785)


In [40]:
#Build sub-table of all rows with low_lam outliers - should be the empty table - no rows
loan_table.loc[loan_table['LoanAmount'] < low_lam]

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status


In [41]:
#Build sub-table of all rows with high_lam outliers - should have 14 rows
loan_table.loc[loan_table['LoanAmount'] > high_lam]

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
130,Male,No,0,Graduate,Yes,20166,0.0,650.0,480.0,NaN,Urban,Y
155,Male,Yes,3+,Graduate,No,39999,0.0,600.0,180.0,0.0,Semiurban,Y
171,NaN,Yes,3+,Graduate,No,51763,0.0,700.0,300.0,1.0,Urban,Y
177,Male,Yes,3+,Graduate,No,5516,11300.0,495.0,360.0,0.0,Semiurban,N
278,Male,Yes,0,Graduate,No,14583,0.0,436.0,360.0,1.0,Semiurban,Y
308,Male,No,0,Graduate,No,20233,0.0,480.0,360.0,1.0,Rural,N
333,Male,Yes,0,Graduate,NaN,63337,0.0,490.0,180.0,1.0,Urban,Y
369,Male,Yes,0,Graduate,No,19730,5266.0,570.0,360.0,1.0,Rural,N
432,Male,No,0,Graduate,NaN,12876,0.0,405.0,360.0,1.0,Semiurban,Y
487,Male,Yes,1,Graduate,No,18333,0.0,500.0,360.0,1.0,Urban,N


What looks suspicious to me are rows 177 and 523 with low `ApplicantIncome`. Unlikely someone will ask for a big loan with small income. However, both rows compensate by having `CoapplicantIncome` to fall back on.

# 4. Handle empties (part 1)

Focus on the `LoanAmount` column. First build a new column that shows empties. Call it `no_lam`.

Then build another column and fill empties with mean. Call this new column `filled_lam`.

In [42]:
#build no_lam column
loan_table['no_lam'] = loan_table.apply(lambda row: 1 if pd.isnull(row.LoanAmount) else 0, axis=1)
loan_table.head(1)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y,1


In [43]:
# Build  filled_lam column
mean_lam = loan_table['LoanAmount'].mean()
loan_table['filled_lam'] = loan_table.apply(lambda row: mean_lam if pd.isnull(row.LoanAmount) else row.LoanAmount, axis=1)
loan_table.head(1)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y,1,146.412162


# 4. Handle empties (part 2)

Now focus on `Property_Area`. Use one-hot encoding to generate 4 new columns. Why 4? Because there are empties in this column. Prefix the columns with `pa`.

In [44]:
#do one-hot encoding to get 4 new columns
one_hot_pa = pd.get_dummies(loan_table['Property_Area'],prefix='pa',dummy_na=True)
loan_table = loan_table.join(one_hot_pa)
loan_table.head(1)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y,1,146.412162,0,0,1,0


# 5. Bin continuous columns

Focus on `filled_lam`. Create a new column `lam_bin` that bins the values into 3 bins Low, Average, High. Allow the `cut` operator to choose the boundaries for you.

In [45]:
#add the new column that contains the 3 bins.
bins = 3  # will divide into 3 evenly spaced bins
bin_names = ['Low', 'Average', 'High']
loan_table['lam_bin'] = pd.cut(loan_table['filled_lam'], bins, labels=bin_names)

loan_table['lam_bin'].value_counts()

Low        556
Average     46
High        12
Name: lam_bin, dtype: int64

# 6. One more one-hot encoding

Use one-hot encoding on `lam_bin`. You do *not* need to include an empty column given you have already captured that info in no_lam. So you should have 3 new columns generated.

In [46]:
# do ohe on the lam_bin column
# loan_table = loan_table.drop(['lam_Low'], axis=1)
# loan_table = loan_table.drop(['lam_Average'], axis=1)
# loan_table = loan_table.drop(['lam_High'], axis=1)
# loan_table = loan_table.drop(['lam_nan'], axis=1)

one_hot_pa = pd.get_dummies(loan_table['lam_bin'],prefix='lam',dummy_na=False)
loan_table = loan_table.join(one_hot_pa)

loan_table.head(1)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,lam_bin,lam_Low,lam_Average,lam_High
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y,1,146.412162,0,0,1,0,Low,1,0,0


# 7. Write it out

The next assignment will start with what you have done here. So write it out so you can read it back in next assignment.

In [47]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#now write it
with open('/content/gdrive/My Drive/Colab Notebooks/399/notebook_history/loan_wrangled_week1.csv', 'w') as f:
  loan_table.to_csv(f, encoding='utf-8', index=False)